# 1、Import Data

In [1]:
import pandas as pd
import statsmodels.stats.api as sms
from termcolor import colored, cprint
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats as stats

In [23]:
## Import Dataset ##

# read excel
file_path = '/Users/sarah/Desktop/Aeon combine dataset.xlsx'
excel_file = pd.ExcelFile(file_path)

# create empty dataframe
combined_df = pd.DataFrame()

# read each sheet and combined to combined_df
for sheet_name in excel_file.sheet_names:
    # read each sheet first cell
    first_cell = pd.read_excel(excel_file, sheet_name=sheet_name, nrows=1, header=None).iloc[0, 0]
    
    # read "Non Salesday" and "非繁忙時段"
    sales_day = 'No' if 'Non Salesday' in str(first_cell) else 'Yes'
    peak = 'No' if '非繁忙時段' in str(first_cell) else 'Yes'
    
    # read sheet data from the second row
    df = pd.read_excel(excel_file, sheet_name=sheet_name, header=1)
    
    # add PEAK and SALES_DAY columns
    df['PEAK'] = peak
    df['SALES_DAY'] = sales_day
    
    # combined all datasets
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# creat a new combined excel
output_file_path = 'Aeon combine dataset-2.xlsx'
combined_df.to_excel(output_file_path, index=False)

In [18]:
combined_df

,門店編碼,銷售單號(N),DEPARTMENT名稱,SECTION,SECTION名稱,商品名稱,貨號 (N),銷售價,數量,會員ID (N),PEAK,SALES_DAY
0,1,****9038854327,HOUSEHOLD & EA,165,LIVING APPLIANCE,TURBO 男士鼻毛剪套裝 THT200,***30522908,208.0,1,*******2027995,Yes,Yes
1,1,****9038864013,HOUSEHOLD & EA,165,LIVING APPLIANCE,伊瑪子母燉盅 ISP16,***00769141,290.0,1,NaN,Yes,Yes
2,1,****9038867517,FASHION,118,UNDERWEAR,TV158SA5303H PFWM綿 男裝V領長袖內衣 BK L,***12490298,79.0,1,*******1758861,Yes,Yes
3,1,****9038867517,FASHION,118,UNDERWEAR,TV158SA5403H PFWM綿 男裝圓領長袖內衣 CGY L,***12490488,79.0,1,*******1758861,Yes,Yes
4,1,****9038880116,FOOD & DELICA,133,DELICA,炸蒜照燒雞扒便當,***00405852,39.9,1,NaN,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
87615,12,****8985588706,FOOD & DELICA,131,GROCERY,福字 米粉5包裝,***02900124,22.9,1,NaN,No,Yes
87616,12,****8985658528,FOOD & DELICA,134,DAILY,榮鈦行 糯米雞,***02036077,38.9,1,NaN,No,Yes
87617,12,****8985658528,FOOD & DELICA,131,GROCERY,SURF BEVERAGE 香印提子綠茶,***20486981,9.9,1,NaN,No,Yes
87618,12,****8985720217,HOUSEHOLD & EA,171,NON FOOD,維達4D DELUXE袋裝面紙5包裝(天然無香),***30270300,27.5,1,NaN,No,Yes


In [102]:
## Adding Member & SALES column ##

# define member or non-member with a new column "MEMBER"
member = []
for i in range(len(combined_df)):
    if pd.isna(combined_df["會員ID (N)"][i]) == True:
        member.append("No")
    else:
        member.append("Yes")
        
combined_df["MEMBER"] = member

# calculating the sum of product sales for each transaction "SALES"
sales = []
for i in range(len(combined_df)):
    product_sales = combined_df['銷售價'][i]*combined_df['數量'][i]
    sales.append(product_sales)
    
combined_df["SALES"] = sales

# add PB column 
pb = []
for i in range(len(combined_df)):
    if combined_df['商品名稱'][i].startswith('TV') == True:
        pb.append("PB")
    else:
        pb.append("Non-PB")
combined_df["PB"] = pb

combined_df

,門店編碼,銷售單號(N),DEPARTMENT名稱,SECTION,SECTION名稱,商品名稱,貨號 (N),銷售價,數量,會員ID (N),PEAK,SALES_DAY,MEMBER,SALES,PB
0,1,****9038854327,HOUSEHOLD & EA,165,LIVING APPLIANCE,TURBO 男士鼻毛剪套裝 THT200,***30522908,208.0,1,*******2027995,Yes,Yes,Yes,208.0,Non-PB
1,1,****9038864013,HOUSEHOLD & EA,165,LIVING APPLIANCE,伊瑪子母燉盅 ISP16,***00769141,290.0,1,NaN,Yes,Yes,No,290.0,Non-PB
2,1,****9038867517,FASHION,118,UNDERWEAR,TV158SA5303H PFWM綿 男裝V領長袖內衣 BK L,***12490298,79.0,1,*******1758861,Yes,Yes,Yes,79.0,PB
3,1,****9038867517,FASHION,118,UNDERWEAR,TV158SA5403H PFWM綿 男裝圓領長袖內衣 CGY L,***12490488,79.0,1,*******1758861,Yes,Yes,Yes,79.0,PB
4,1,****9038880116,FOOD & DELICA,133,DELICA,炸蒜照燒雞扒便當,***00405852,39.9,1,NaN,Yes,Yes,No,39.9,Non-PB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87615,12,****8985588706,FOOD & DELICA,131,GROCERY,福字 米粉5包裝,***02900124,22.9,1,NaN,No,Yes,No,22.9,Non-PB
87616,12,****8985658528,FOOD & DELICA,134,DAILY,榮鈦行 糯米雞,***02036077,38.9,1,NaN,No,Yes,No,38.9,Non-PB
87617,12,****8985658528,FOOD & DELICA,131,GROCERY,SURF BEVERAGE 香印提子綠茶,***20486981,9.9,1,NaN,No,Yes,No,9.9,Non-PB
87618,12,****8985720217,HOUSEHOLD & EA,171,NON FOOD,維達4D DELUXE袋裝面紙5包裝(天然無香),***30270300,27.5,1,NaN,No,Yes,No,27.5,Non-PB


# 2、Sales Analysis

In [103]:
# sales of PB and Non-PB goods
combined_df.groupby(["PB"])["SALES"].sum().to_frame().rename(index={"PB": "PB"})

,SALES
PB,
Non-PB,5034204.5
PB,309577.9


In [105]:
PBsales_ratio = round((combined_df.loc[combined_df["PB"]=="PB"]["SALES"].sum() / combined_df["SALES"].sum()) * 100, 2)
print(f"The sales percentage of PB is {PBsales_ratio}%.")

The sales percentage of PB is 5.79%.


## 2.1 Sales analysis by stores

In [110]:
# PB and Non-PB sale quantity across stores
combined_df.groupby(["門店編碼", "PB"]).agg({"數量": "sum"})\
.pivot_table(index="門店編碼", columns="PB", values="數量").rename_axis("SALE QUANTITY", axis=1)

SALE QUANTITY,Non-PB,PB
門店編碼,,
1,27223,1160
2,10473,601
3,2719,195
4,4977,276
5,7860,489
6,5202,366
7,17135,883
8,11881,579
9,9197,347


In [112]:
# PB and Non-PB sale quantity across stores
combined_df.groupby(["門店編碼", "PB"]).agg({"SALES": "sum"})\
.pivot_table(index="門店編碼", columns="PB", values="SALES").rename_axis("SALES", axis=1)

SALES,Non-PB,PB
門店編碼,,
1,1374759.3,63452.9
2,539887.8,35584.2
3,56742.5,6269.5
4,207958.6,17119.1
5,359403.3,28355.2
6,335367.9,23494.4
7,846020.9,47869.6
8,557483.7,38270.4
9,243117.4,14767.1


In [27]:
# total sales in different stores
combined_df.groupby(by = "門店編碼")["SALES"].sum().sort_values(ascending=False).to_frame()

,SALES
門店編碼,
1,1438212.2
7,893890.5
8,595754.1
2,575472.0
5,387758.5
6,358862.3
10,284726.4
9,257884.5
4,225077.7


In [29]:
# ranking the store by sales
combined_df.groupby(by="門店編碼").agg({"數量": "sum", "SALES": "sum"})\
.sort_values(by="SALES", ascending=False)

,數量,SALES
門店編碼,,
1,28383,1438212.2
7,18018,893890.5
8,12460,595754.1
2,11074,575472.0
5,8349,387758.5
6,5568,358862.3
10,5578,284726.4
9,9544,257884.5
4,5253,225077.7


Store 1 has the highest sales $1438212.2, and quantity 28383.

## 2.2 Sales analysis by department

### 2.2.1 General

In [31]:
# sales qunatity in different categories
combined_df.groupby(by = "DEPARTMENT名稱")["數量"].sum().sort_values(ascending = False).to_frame().rename_axis("DEPARTMENT")

,數量
DEPARTMENT,
FOOD & DELICA,75598
HOUSEHOLD & EA,11608
LIVING PLAZA,10651
FASHION,8460
MISC-OTHER SALES,4814
BENTO & BAKERY,3533
MISC-PACKING,2037
MISCELLANEOUS,133
MOLLY FANTASY,1


In [32]:
# sales in different categories
combined_df.groupby(by = "DEPARTMENT名稱")["SALES"].sum().sort_values(ascending=False).to_frame()

,SALES
DEPARTMENT名稱,
FOOD & DELICA,2276133.7
HOUSEHOLD & EA,1605870.3
FASHION,1100530.3
LIVING PLAZA,134819.0
MISC-OTHER SALES,94496.5
MISCELLANEOUS,76267.2
BENTO & BAKERY,53518.4
MISC-PACKING,2037.0
MOLLY FANTASY,98.0


In [33]:
# ranking the dept by sales
combined_df.groupby(by="DEPARTMENT名稱").agg({"數量": "sum", "SALES": "sum"})\
.sort_values(by="SALES", ascending=False)

,數量,SALES
DEPARTMENT名稱,,
FOOD & DELICA,75598,2276133.7
HOUSEHOLD & EA,11608,1605870.3
FASHION,8460,1100530.3
LIVING PLAZA,10651,134819.0
MISC-OTHER SALES,4814,94496.5
MISCELLANEOUS,133,76267.2
BENTO & BAKERY,3533,53518.4
MISC-PACKING,2037,2037.0
MOLLY FANTASY,1,98.0


FOOD & DELICA department has the highest sales and quantity. 

MOLLY FANTASY department has the least sales $98.0, and quantity 1.

### 2.2.2  HOUSEHOLD & EA

#### 2.2.2.1 Quantity VS Sales

In [40]:
# sales quantity in different sections of HOUSEHOLD & EA
combined_df.loc[combined_df["DEPARTMENT名稱"] == "HOUSEHOLD & EA"].groupby(by = "SECTION名稱")["數量"].sum()\
.to_frame().sort_values(by="數量", ascending=False)

,數量
SECTION名稱,
NON FOOD,5523
KITCHEN & DINING,1370
HOUSEKEEPING,1233
COSMETIC,1139
HEALTH,673
BEDDING,661
LIVING APPLIANCE,485
FURNITURE & INTERIOR,295
AUDIO VISUAL,147


In [36]:
# sales in different sections of HOUSEHOLD & EA
combined_df.loc[combined_df["DEPARTMENT名稱"] == "HOUSEHOLD & EA"].groupby(by = "SECTION名稱")["SALES"].sum()\
.to_frame().sort_values(by="SALES", ascending=False)

,SALES
SECTION名稱,
FURNITURE & INTERIOR,354051.1
LIVING APPLIANCE,339063.8
BEDDING,232427.6
NON FOOD,199638.2
KITCHEN & DINING,195223.2
COSMETIC,86651.8
HOUSEKEEPING,81586.0
AUDIO VISUAL,73747.3
HEALTH,34341.1


In "HOUSEHOLD & EA" department, NON FOOD section sold quantity most, but FURNITURE & INTERIOR section has the highest sales.

#### 2.2.2.2 Top 10 products in  HOUSEHOLD & EA

In [42]:
combined_df.loc[combined_df["DEPARTMENT名稱"] == "HOUSEHOLD & EA"].groupby(by="商品名稱")\
.agg({"SECTION名稱": "first", "數量": "sum"}).sort_values(by="數量", ascending=False).head(10)

,SECTION名稱,數量
商品名稱,,
法國雙飛人 藥水,HEALTH,132
思詩樂嬰兒清潔消毒棉,NON FOOD,74
CONSIG.- TAKAGI TR,HOUSEKEEPING,69
高潔絲 純棉面層特長衛生護墊,NON FOOD,66
王子KA 4合1抗菌除蟎洗衣珠袋裝16G*40粒,NON FOOD,62
潔霸超濃縮殺菌洗衣棒 51支裝,NON FOOD,56
CONSIG.- MAIN PLAN,KITCHEN & DINING,56
BESTPRICE BY TOPVALU 軟包紙巾,NON FOOD,54
ADP PENTAGON,HOUSEKEEPING,52


In "HOUSEHOLD & EA" department, 法國雙飛人 藥水 sold quantity most.

#### 2.2.2.3 Top 5 products by section 

In [43]:
# get data when DEPARTMENT = "HOUSEHOLD & EA"
household_df = combined_df[combined_df["DEPARTMENT名稱"] == "HOUSEHOLD & EA"]

# count by each section & item
grouped_df = household_df.groupby(['SECTION名稱', '商品名稱'])['數量'].sum().reset_index()

# get top 5 items by each section
top5_items_df = grouped_df.sort_values(['SECTION名稱', '數量'], ascending=[True, False]) \
                          .groupby('SECTION名稱') \
                          .head(5) \
                          .reset_index(drop=True)

print(top5_items_df)

               SECTION名稱                               商品名稱   數量
0           AUDIO VISUAL                 REE SCHEME(TR)-281   10
1           AUDIO VISUAL           REMAX 多功能數碼線收納盒RC-190 BL    8
2           AUDIO VISUAL          X3D HDMI CABLE HDMI-SS201    8
3           AUDIO VISUAL           REMAX 多功能數碼線收納盒RC-190 WH    7
4           AUDIO VISUAL              X3D RF CABLE RFC-S115    7
5                BEDDING                  CONSIG.- WING YIP   36
6                BEDDING                 CONSIG.- J-TEX (HK   32
7                BEDDING                 CONSIG.- PANG TANG   31
8                BEDDING        CONSIG.- CASABLANCA BEDDING   28
9                BEDDING                   雅芳婷19"X29"抗病毒珍珠枕   25
10              COSMETIC                    小久保即棄洗面紙 - 極細纖?   25
11              COSMETIC                           麗的 化粧棉孖裝   23
12              COSMETIC               BIFESTA 速效卸妝潔膚紙(保濕型)   12
13              COSMETIC                         水清新葡萄籽保濕乳液   12
14              COSMETIC 

In [64]:
def get_top_products(group):
    return combined_df.loc[combined_df["SECTION名稱"] == group].groupby(by = "商品名稱")["數量"].sum()\
.to_frame().sort_values(by="數量", ascending=False).head(5)

In [67]:
section_names = household_df["SECTION名稱"].unique().tolist()
print(section_names)

['LIVING APPLIANCE', 'COSMETIC', 'KITCHEN & DINING', 'NON FOOD', 'BEDDING', 'HOUSEKEEPING', 'HEALTH', 'FURNITURE & INTERIOR', 'NATURAL', 'AUDIO VISUAL']


In [68]:
get_top_products("LIVING APPLIANCE")

,數量
商品名稱,
蠟筆小新 2A 鹼性電池 8粒裝,16
WINSTAR 3位電源萬能蘇 WW07G3CPD WH,14
GIFT LEO F/STEAMER KS72,12
飛利浦濾水器套裝 WP3812+WP3922X2,11
DOULTON FLYER PROMOTION,9


In [69]:
get_top_products("COSMETIC")

,數量
商品名稱,
小久保即棄洗面紙 - 極細纖?,25
麗的 化粧棉孖裝,23
水清新葡萄籽保濕乳液,12
BIFESTA 速效卸妝潔膚紙(保濕型),12
CUREL柔潤保濕身體乳霜90G,10


In [70]:
get_top_products("KITCHEN & DINING")

,數量
商品名稱,
CONSIG.- MAIN PLAN,56
CONSIG.- MYREX INT,21
IDEAL CHEF PROMO,16
TVHC 陶瓷茶杯連碟 (6/36),16
LOCK'N LOCK PLASTIC MICROWAVE BOX,14


In [65]:
get_top_products("NON FOOD")

,數量
商品名稱,
思詩樂嬰兒清潔消毒棉,74
高潔絲 純棉面層特長衛生護墊,66
王子KA 4合1抗菌除蟎洗衣珠袋裝16G*40粒,62
潔霸超濃縮殺菌洗衣棒 51支裝,56
BESTPRICE BY TOPVALU 軟包紙巾,54


In [71]:
get_top_products("BEDDING")

,數量
商品名稱,
CONSIG.- WING YIP,36
CONSIG.- J-TEX (HK,32
CONSIG.- PANG TANG,31
CONSIG.- CASABLANCA BEDDING,28
"雅芳婷19""X29""抗病毒珍珠枕",25


In [72]:
get_top_products("HOUSEKEEPING")

,數量
商品名稱,
CONSIG.- TAKAGI TR,69
ADP PENTAGON,52
TVHC 除塵濕紙20片裝 (72),23
TVHC 除塵乾紙30片裝 (54),16
TVHC 減少衣領變形衣架 (60),15


In [73]:
get_top_products("HEALTH")

,數量
商品名稱,
法國雙飛人 藥水,132
小林小白兔 暖身貼,30
小林小白兔 迷你暖身貼,19
TVBP 暖貼(標準) 30個裝,18
TVBP 暖包(標準) 30個裝,17


In [74]:
get_top_products("FURNITURE & INTERIOR")

,數量
商品名稱,
HOP HING寢室木器傢俬,11
IRIS塑料可疊式有蓋儲物箱 大/白色 (16),6
TVHC 不織布籃-杏(10),5
TVHC HEAT 燈芯絨寵物床35×45厘米 淺黃 (4),5
MY FAMILY 透明手提收納箱 6L(45),5


In [75]:
get_top_products("NATURAL")

,數量
商品名稱,
THE GOAT 護膚肥皂 - 椰子,8
繽紛滋潤禮盒,3
RENE FURTERER豐盈亮澤洗髮水 250ML,3
BURT’S BEES 積雪草萬用軟膏筆,3
KLORANE 有機牡丹抗敏紓緩護髮素 150ML,2


In [76]:
get_top_products("AUDIO VISUAL")

,數量
商品名稱,
REE SCHEME(TR)-281,10
X3D HDMI CABLE HDMI-SS201,8
REMAX 多功能數碼線收納盒RC-190 BL,8
X3D RF CABLE RFC-S115,7
REMAX 多功能數碼線收納盒RC-190 WH,7


### 2.2.3 FOOD & DELICA

#### 2.2.3.1 Quantity VS Sales

In [77]:
# sales quantity in different sections of FOOD & DELICA
combined_df.loc[combined_df["DEPARTMENT名稱"] == "FOOD & DELICA"].groupby(by = "SECTION名稱")["數量"].sum()\
.to_frame().sort_values(by="數量", ascending=False)

,數量
SECTION名稱,
GROCERY,25541
DAILY,21247
PRODUCE,12795
DELICA,5947
MEAT,5166
FISH,3795
LIQUOR,1107


In [78]:
# sales in different sections of FOOD & DELICA
combined_df.loc[combined_df["DEPARTMENT名稱"] == "FOOD & DELICA"].groupby(by = "SECTION名稱")["SALES"].sum()\
.to_frame().sort_values(by="SALES", ascending=False)

,SALES
SECTION名稱,
GROCERY,575180.2
DAILY,500713.1
PRODUCE,310716.4
MEAT,306342.9
FISH,263538.1
DELICA,250999.9
LIQUOR,68643.1


In "FOOD & DELICA" department, GROCERY section sold quantity most, and also the sales.

#### 2.2.3.2 Top 10 products in FOOD & DELICA

In [79]:
combined_df.loc[combined_df["DEPARTMENT名稱"] == "FOOD & DELICA"].groupby(by="商品名稱")\
.agg({"SECTION名稱": "first", "數量": "sum"}).sort_values(by="數量", ascending=False).head(10)

,SECTION名稱,數量
商品名稱,,
日本之惠 白蛋 10隻裝,DAILY,822
澳洲橙56庄,PRODUCE,544
澳洲 SWEETEST BATCH 珍寶藍莓,PRODUCE,537
燒味精選,DELICA,443
TOPVALU 香蕉 (越南產),PRODUCE,442
TOPVALU 北海道十勝?雪靜小粒納豆,DAILY,342
北海道3.6牛乳(迷你裝),DAILY,328
日清 咚兵衛腐皮烏冬,GROCERY,310
日本福岡秋香園本菇,PRODUCE,307


#### 2.2.3.3 Top 5 products by section

In [80]:
# Get data when DEPARTMENT = "HOUSEHOLD & EA"
F_D_df = combined_df[combined_df["DEPARTMENT名稱"] == "FOOD & DELICA"]

section_names = F_D_df["SECTION名稱"].unique().tolist()
print(section_names)

['DELICA', 'PRODUCE', 'GROCERY', 'DAILY', 'MEAT', 'FISH', 'LIQUOR']


In [81]:
get_top_products("GROCERY")

,數量
商品名稱,
日清 咚兵衛腐皮烏冬,310
WAIWAI 泰國米粉(肉碎味),220
維他純蒸餾水,160
西班牙黑毛豬午餐肉,148
農心 薯仔麵4包裝,143


In [82]:
get_top_products("DAILY")

,數量
商品名稱,
日本之惠 白蛋 10隻裝,822
TOPVALU 北海道十勝?雪靜小粒納豆,342
北海道3.6牛乳(迷你裝),328
蘭王雞蛋 10隻裝,258
大阪王將 餃子(羽根),208


### 2.2.4 LIVING PLAZA

#### 2.2.4.1 Quantity VS Sales

In [86]:
# sales quantity in different sections of LIVING PLAZA
combined_df.loc[combined_df["DEPARTMENT名稱"] == "LIVING PLAZA"].groupby(by = "SECTION名稱")["數量"].sum()\
.to_frame().sort_values(by="數量", ascending=False)

,數量
SECTION名稱,
LIVING PLAZA,10651


In [87]:
# sales in different sections of LIVING PLAZA
combined_df.loc[combined_df["DEPARTMENT名稱"] == "LIVING PLAZA"].groupby(by = "SECTION名稱")["SALES"].sum()\
.to_frame().sort_values(by="SALES", ascending=False)

,SALES
SECTION名稱,
LIVING PLAZA,134819.0


#### 2.2.4.2 Top 10 products in LIVING PLAZA

In [88]:
combined_df.loc[combined_df["DEPARTMENT名稱"] == "LIVING PLAZA"].groupby(by="商品名稱")\
.agg({"SECTION名稱": "first", "數量": "sum"}).sort_values(by="數量", ascending=False).head(10)

,SECTION名稱,數量
商品名稱,,
Y23 XMAS PEN,LIVING PLAZA,321
BLEND MILD COFFEE CAN,LIVING PLAZA,87
Y23-3 CHRISTMAS CRYSTAL STICKER,LIVING PLAZA,59
Y23 XMAS STAMP,LIVING PLAZA,58
Polyethylene Gloves -L - 120 pcs.-,LIVING PLAZA,51
TEA FILTER BAG 92PIECES WITH A HANGING MOUNT,LIVING PLAZA,45
OILY GRIME CLEANER -20 SHEETS-,LIVING PLAZA,45
Y23-2 CHRISTMAS BRONZING STICKER,LIVING PLAZA,34
Y23 XMAS A5 MESH BAG SET,LIVING PLAZA,32


#### 2.2.4.3 Top 5 products by section

In [89]:
# Get data when DEPARTMENT = "LIVING PLAZA"
F_D_df = combined_df[combined_df["DEPARTMENT名稱"] == "LIVING PLAZA"]

section_names = F_D_df["SECTION名稱"].unique().tolist()
print(section_names)

['LIVING PLAZA']


In [90]:
get_top_products("LIVING PLAZA")

,數量
商品名稱,
Y23 XMAS PEN,321
BLEND MILD COFFEE CAN,87
Y23-3 CHRISTMAS CRYSTAL STICKER,59
Y23 XMAS STAMP,58
Polyethylene Gloves -L - 120 pcs.-,51


### 2.2.5 FASHION

#### 2.2.5.1 Quantity VS Sales

#### 2.2.5.2 Top 10 products in FASHION

#### 2.2.5.3 Top 5 products by section

## 2.3 Membership Rate

### 2.3.1 General

In [92]:
# number of memebers and non-members
num_mem = combined_df.loc[combined_df["會員ID (N)"].notnull(), "銷售單號(N)"].nunique()
num_nonmem = combined_df.loc[combined_df["會員ID (N)"].isnull(), "銷售單號(N)"].nunique()
pd.DataFrame({"Category": ["Member", "Non-member"], "Count": [num_mem, num_nonmem]})

,Category,Count
0,Member,8670
1,Non-member,17341


In [94]:
# count the number of unique invoices with "MEMBER" value
num_members = combined_df.loc[combined_df["會員ID (N)"].notnull(), "銷售單號(N)"].nunique()

# count the total number of unique invoices
total_invoices = combined_df["銷售單號(N)"].nunique()

# calculate the membership rate
membership_rate = (num_members / total_invoices) * 100

# print the membership rate
print(f"Membership rate: {membership_rate:.2f}%")

Membership rate: 33.56%


In [96]:
avg_mem = combined_df.loc[combined_df["MEMBER"] == "Yes"]["SALES"].sum() / num_mem
print(f"The average sales of member is {avg_mem}.")

The average sales of member is 261.22815455593997.


In [98]:
avg_nonmem = combined_df.loc[combined_df["MEMBER"] == "No"]["SALES"].sum() / num_nonmem
print(f"The average sales of non-member is {avg_nonmem}.")

The average sales of non-member is 177.5522922553486.


In [99]:
std1 = np.std(combined_df.loc[combined_df["MEMBER"] == "Yes", "SALES"])
std2 = np.std(combined_df.loc[combined_df["MEMBER"] == "No", "SALES"])

# perform the t-test
t_stat, p_value = stats.ttest_ind_from_stats(avg_mem, std1, num_mem, avg_nonmem, std2, num_nonmem)

# print the t-statistic and p-value
print("T-statistic:", t_stat)
print("P-value:", p_value)

T-statistic: 27.589781455846634
P-value: 3.5305915939700983e-165


From the p-test, there is a statistical difference in sales between members and non-members group at 5% significance level.

### 2.3.2 Membership rate and sales across stores

### 2.3.3 Membership rate and sales across departments

### 2.3.4 Top section in HOUSEHOLD & EA for member

## 2.4 Sales day vs. Non-sales day

### 2.4.1 General

In [100]:
combined_df.groupby(["SALES_DAY"]).agg({"SALES": "sum"})

,SALES
SALES_DAY,
No,1350387.4
Yes,3993395.0


### 2.4.2 Sales ratio by store

### 2.4.3 Sales ratio by departments

### 2.4.4 Membership rate difference in sales day and non sales day